In [35]:
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_csv('bq_data.csv')
df = df.loc[df['receiver'].notnull()]
display(df)
print(df.info())

,date,commit_date,receiver,amount,category,commit_file_id
1,2023-04-20,2023-05-03,H M,-39.99,HOUSEHOLD-ITEMS,POP
2,2023-04-24,2023-05-03,STADIUM,-66.48,HOBBIES,POP
3,2023-04-16,2023-05-03,KLARNA AB,-43.94,HOUSEHOLD-ITEMS,POP
4,2023-04-12,2023-05-03,KLARNA AB,-221.30,UNCATEGORIZED,POP
5,2023-04-03,2023-05-03,CHF*PNTpay,-5.00,ENTERTAINMENT,POP
...,...,...,...,...,...,...
1188,2023-03-06,2023-04-04,SEPPÄLÄ SEIJA TUULIKKI,25.00,OTHER-INCOME,POP
1189,2023-03-21,2023-04-04,IKEA VANTAA IF CUSTOMER R,-9.68,FOOD,POP
1190,2023-03-20,2023-04-04,K-CITYMARKET ESPOO LEPPAV,-75.68,FOOD,POP
1191,2023-03-10,2023-04-04,MOBILEPAY KATARIINA SEPPÄLÄ,100.00,OTHER-INCOME,POP


<class 'pandas.core.frame.DataFrame'>
Index: 1141 entries, 1 to 1192
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   date            1141 non-null   object 
 1   commit_date     1141 non-null   object 
 2   receiver        1141 non-null   object 
 3   amount          1141 non-null   float64
 4   category        1130 non-null   object 
 5   commit_file_id  1141 non-null   object 
dtypes: float64(1), object(5)
memory usage: 62.4+ KB
None


In [119]:
from time import time
def my_timing(func):
    def wrapper(*args, **kwargs):
        t0 = time()
        values = func(*args, **kwargs)
        t1 = time()
        print(f'Function: {func.__name__}')
        print(f'With args: [{args}, {kwargs}]')
        print(f'Took {t1 - t0:.6f}s to run.\n')
        return values
    return wrapper

In [28]:
def encode_variables(data: np.ndarray) -> dict:
    assert len(data.shape) == 1
    return {i: label for i, label in enumerate(np.unique(data))}

In [38]:
@my_timing
def transform_y(y):
    return encode_variables(y)


y = df['category'].to_list()
y = np.asarray(y)
labels = transform_y(y)
display(labels)

Function: transform_y
With args: [(array(['HOUSEHOLD-ITEMS', 'HOBBIES', 'HOUSEHOLD-ITEMS', ..., 'FOOD',
       'OTHER-INCOME', 'OTHER-INCOME'], dtype='<U32'),), {}]
Took 0.003s to run.



{0: 'CLOTHING',
 1: 'COMMUTING',
 2: 'ENTERTAINMENT',
 3: 'FOOD',
 4: 'HEALTH',
 5: 'HOBBIES',
 6: 'HOUSEHOLD-ITEMS',
 7: 'INVESTING',
 8: 'LIVING',
 9: 'OTHER-INCOME',
 10: 'SALARY',
 11: 'TECHNOLOGY',
 12: 'UNCATEGORIZED',
 13: 'nan'}

In [121]:
@my_timing
def learn_vocabulary(X):
    X = [re.split(';|,|-|_|\*|\s', str(row).lower()) for row in X]
    X = set(value for nested_list in X for value in nested_list)
    return X


""" x = np.array([1, 2, 3, 4, 5])
f = lambda x: x ** 2
squares = f(x) """


X = df['receiver'].to_list()
X = learn_vocabulary(X)
display(X)

Function: learn_vocabulary
With args: [(['H M', 'STADIUM', 'KLARNA AB', 'KLARNA AB', 'CHF*PNTpay', 'HSL Mobiili', 'HSL Mobiili', 'HSL Mobiili', 'HSL Mobiili', 'HSL Mobiili', 'VEIKKAUS OY', 'DNA Welho Oy', 'VR-YHTYMÄ OY', 'Vattenfall Oy', 'APPLE.COM/BILL', 'TOPI FEELGOOD K', 'VALKOINEN PUU C', 'VFI*VEIKKAUS OY', 'VFI*VEIKKAUS OY', 'VFI*VEIKKAUS OY', 'VFI*VEIKKAUS OY', 'VFI*VEIKKAUS OY', 'VFI*VEIKKAUS OY', 'VFI*VEIKKAUS OY', 'VFI*VEIKKAUS OY', 'VFI*VEIKKAUS OY', 'VFI*VEIKKAUS OY', 'VFI*VEIKKAUS OY', 'VFI*VEIKKAUS OY', 'VERKKOKAUPPA.COM', 'VERKKOKAUPPA.COM', 'RASMUS HAAPANIEMI', 'STOCKMANN TAPIOLA', 'SUOMEN OSUUSPANKKI', 'CHF*Krkkinen Web Oy', 'K-Supermarket Derby', 'K-Supermarket Derby', 'K-Supermarket Derby', 'K-Supermarket Derby', 'K-Supermarket Derby', 'KLARNA BANK AB PUBL', 'VERIFONE FINLAND OY', 'A MEDIUM CORPORATION', 'ADOBE PHOTOGPHY PLAN', 'ALEPA KIVISTON ASEMA', 'CENTAURUS FINLAND KY', 'KATJAN VOHVELIKAHVILA', 'Tekniikan akateemiset', 'K-citymarket Kauhajoki', 'Kauhajoen Ege Keb

{'',
 '&',
 '00520',
 '044',
 '0448',
 '0786',
 '10.',
 '1226',
 '1280',
 '207',
 '21',
 '217',
 '236',
 '257',
 '26712',
 '28709',
 '2e0pe6im4',
 '2r4ea2eg4',
 '3',
 '3071',
 '35',
 '358753257553',
 '3650',
 '4',
 '40',
 '4155085008',
 '41596',
 '440',
 '50',
 '57',
 '57389',
 '5780',
 '60',
 '6157',
 '65975',
 '66',
 '7',
 '7038',
 '704',
 '707',
 '712',
 '716',
 '740',
 '75',
 '7802',
 '928',
 'a',
 'aal',
 'aalto',
 'aaltoyo',
 'ab',
 'abb',
 'abc',
 'adguard',
 'adobe',
 'adomex',
 'ahti',
 'ainoa',
 'akateeminen',
 'akateemiset',
 'aleksi',
 'alepa',
 'alkanlar',
 'alko',
 'am',
 'amazon.de',
 'amos',
 'amzn',
 'antti',
 'apple.com/bill',
 'apteekk',
 'apteekki',
 'arima',
 'arkadia',
 'arlanda',
 'arn',
 'as',
 'as.',
 'asem',
 'asema',
 'at',
 'aurinkolah',
 'avecra',
 'axa',
 'b.v',
 'bank',
 'baoyu',
 'benjamin',
 'bestseller',
 'bikecompone',
 'biltema.fi',
 'bocholt',
 'bowlcircus',
 'bu',
 'buffa',
 'burgeri',
 'c',
 'ca778475780',
 'cafe',
 'caruna',
 'casino',
 'centauru

In [122]:
@my_timing
def in_test(val, X):
    if val in X: 
        return True
    else: 
        return False

in_test(val='ei oo', X=X)

Function: in_test
With args: [(), {'val': 'ei oo', 'X': {'', 'rav', 'oyj', 'heslsinki', 'rauta', 'hemtex', 'amzn', 'espoon', 'kaupunki', 'im', 'helsi', '4', 'hsl', 'antti', 'helsingin', 'holy', 'enterpri', 'saara', 'derby', 'jumbo', 'danai', 'kir', 'adguard', 'laivalahdenportti', 'tapiola', 'v', 'bikecompone', 'jededudet', 'vuosa', '712', 'vattenfall', 'laitaan', 'teepakki', 'seija', 'johanna', 'columbus', 'o', 'rex', 'walk', 'sanna', 'outlet', 'easton', '217', 'lounas', 'kalalahti', 'pntpay', '3650', 'ecomm', 'sornaine', 'cafe', 'seinaejoki', 'apteekki', '40', 'dna', 'kahvikaveri', 'oy/aseman', 'gigantti', '21', 'photogphy', 'b.v', 'mai', '4155085008', 'j', 'vuosaari', 'takanen', 'saiturinporssi', 'suitsupply', 'fotoyks', 'taste', 'maarit', 'tower', 'lasipalatsi', 'niittykumpu', 'international', 'redi', 'pop', 'vohvelikahvila', 'vuosaaren', 'connell', 'ainoa', 'adomex', 'partioaitta', 'baoyu', 'kannelmaki', 'prisma', 'fitnessgym', 'helene', 'saariselka', 's', 'kiasma', 'sport', '66', 

False